# 다음의 News Classification Using FastText

In [19]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob
import warnings 
from numba import jit

os.environ['KERAS_BACKEND']='tensorflow'

import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import FastText, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score, auc
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler, LabelEncoder
from sklearn.manifold import TSNE
from sklearn.multiclass import OneVsRestClassifier

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt


import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

In [2]:
print (device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12801240253654533557
]


In [3]:
import Basic_Module as bm

## Load Data

In [4]:
#Daum
daumData = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
daumData = pd.DataFrame.from_dict(daumData, orient = 'index')
daumData.reset_index(inplace = True)
daumData.rename(columns = {'index' : 'id'}, inplace = True)
extDaumData = daumData.loc[:,['id','title','extracted_keywords']].copy()
print ('Daum : {}'.format(daumData.shape))

Daum : (9372, 11)


## Stopwords

In [5]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## Document Labeling

In [30]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags category')

## Category

In [7]:
if os.path.isfile('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled'):
    le = pickle.load(open('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled','rb'))
else:
    le = LabelEncoder()
    le.fit(naverData['category'])
    pickle.dump(le, open('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled','wb'))
print (le.classes_)

['IT/과학' '경제' '사회' '생활/문화' '세계' '스포츠' '연예' '정치']


In [8]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/news_model/'
elif sys.platform =='win32':
    loadModelPath = 'd:/news_model/'
daumNewsPath = './data/pre_data/news_daum_news/'
classifierPath = './data/pre_data/news_classifier/'

In [9]:
daumData.head()

,id,category,date,press,number_of_comment,number_of_crawled_comment,rank,title,mainText,keywords,extracted_keywords
0,5a2a61bf588c13481c229d1e,뉴스,2017.12.07,세계일보,1093,911,1,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, ...","[부작용, 비아그라, 아일랜드]","{세보 효과, 지역, 남성들, 건강, 부작용, 연기, 공장}"
1,5a2a61bf588c13481c229d1f,뉴스,2017.12.07,헬스조선,603,386,2,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,점심식사를 마친 후 후식으로 커피를 마시는 사람들이 많다. 실제로 직장이 밀집돼 ...,"[커피, 낮잠, 음식물]","{식후, 디스크, 낮잠, 치아, 자세, 입냄새, 건강, 커피, 점심 식사, 철분}"
2,5a2a61bf588c13481c229d20,뉴스,2017.12.07,연합뉴스,1067,811,3,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""",(성남=연합뉴스) 최해민 기자 = 십년지기 지인을 산 채로 묻어 살해한 50대 여...,"[살인혐의, 철원, 검찰송치]","{철원, 진술, 아들, 앙심, 경찰, 성관계, 범행, 남편, 지인, 주변}"
3,5a2a61bf588c13481c229d21,뉴스,2017.12.07,헤럴드경제,418,369,4,"신영자, 억 소리나는 갑질","신영자, 적용안된 혐의→검찰 상고에서 인정\n신영자, 얼마를 어떻게 받았나 [헤럴...","[신영자, 갑질, 롯데백화점]","{징역, 매장, 네이처리퍼블릭, 혐의, 롯데, 신영자 이사장, 유통업체, 검찰}"
4,5a2a61bf588c13481c229d22,뉴스,2017.12.07,연합뉴스,434,368,5,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지",(청주=연합뉴스) 이승민 기자 = 지난 9월 청주의 한 하천에서 20대 여성을 둔기...,"[공범, 살인, 과자]","{남자친구, 폭행, 쪽지, 경찰, 유치장, 혐의, 범행, 과자}"


In [10]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Train data set으로 부터 TF-IDF Vectorizer을 만듦

In [11]:
trainName = './data/pre_data/news_train_test_Data/pre_data_word2vec_train_for_news_classification_by_mecab.pickled'
train = pickle.load(open(trainName, 'rb'))
tfidf = bm.Build_tfidf(train)
del train

100%|██████████| 12852/12852 [00:00<00:00, 1206202.62it/s]


(12852, 73416)
vocab size : 73416


## Word2Vec Model

### Twitter

#### News to Tagged Document

In [12]:
if os.path.isfile('./data/pre_data/news_tagged_data/pre_data_daum_news_by_ct_for_fastText_news_classification.pickled'):
    daumData2 = pickle.load(open('./data/pre_data/news_tagged_data/pre_data_daum_news_by_ct_for_fastText_news_classification.pickled', 'rb'))
else:
    daumData2 = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, ct, stopwords, 'daum')
    pickle.dump(daumData2, open('./data/pre_data/news_tagged_data/pre_data_daum_news_by_ct_for_fastText_news_classification.pickled', 'wb'))

100%|██████████| 9372/9372 [14:19<00:00, 10.91it/s]


#### Load Model

In [13]:
model1 = FastText.load(loadModelPath+'fastText_size-500_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-ct.model')
model2 = FastText.load(loadModelPath+'fastText_size-500_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-ct.model')
model3 = FastText.load(loadModelPath+'fastText_size-500_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-ct.model')

#### Model1

In [14]:
wv1, vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 500, daumData2)
modelName = bm.Return_ModelName('fastText', model1,'ct')

  3%|▎         | 2649/80610 [00:00<00:02, 26482.83it/s]

FastText(vocab=80610, size=500, alpha=0.025)


100%|██████████| 80610/80610 [00:01<00:00, 60756.39it/s]
0it [00:00, ?it/s]

running time : 0:00:01.348332
Vectorizing Data


9372it [04:46, 32.76it/s]


scaling Data
total running time : 0:04:47.701595


In [15]:
classifierList = glob(classifierPath+'*'+modelName)

In [16]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

SVC
NeuralNetwork_1
XGBoost
LogisticRegression
RandomForestClassifier
NeuralNetwork_2


In [20]:
%%time
warnings.filterwarnings('ignore')
predictOutcome = dict(map(lambda x: bm.PredictNewsClassification(vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome = pd.DataFrame.from_dict(predictOutcome)
predictOutcome = predictOutcome.applymap(lambda x: le.inverse_transform(int(x)))
predictOutcome = extDaumData.merge(predictOutcome,
                                   left_index = True, right_index = True)
predictOutcome.to_csv('./outcome/outcome_news_classification_'+modelName,index=None, encoding='utf-8')

4130it [02:00, 34.31it/s]
0it [00:00, ?it/s]
9372it [00:00, 619957.37it/s]
0it [00:00, ?it/s]
9372it [00:00, 695931.89it/s]
0it [00:00, ?it/s]
9372it [00:00, 622884.85it/s]

CPU times: user 42.9 s, sys: 743 ms, total: 43.6 s
Wall time: 43.1 s


#### Model2

In [21]:
wv1, vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 500, daumData2)
modelName = bm.Return_ModelName('fastText', model2,'ct')


  9%|▊         | 7005/80610 [00:00<00:01, 68263.68it/s]

FastText(vocab=80610, size=500, alpha=0.025)



100%|██████████| 80610/80610 [00:00<00:00, 83030.52it/s]
0it [00:00, ?it/s]
4it [00:00, 36.61it/s]

running time : 0:00:00.976118
Vectorizing Data



8it [00:00, 36.84it/s]
13it [00:00, 34.84it/s]
17it [00:00, 34.36it/s]
20it [00:00, 33.29it/s]
25it [00:00, 35.03it/s]
30it [00:00, 35.99it/s]
34it [00:00, 34.43it/s]
38it [00:01, 34.45it/s]
42it [00:01, 33.99it/s]
47it [00:01, 34.67it/s]
51it [00:01, 34.42it/s]
58it [00:01, 36.03it/s]
64it [00:01, 37.18it/s]
69it [00:01, 37.53it/s]
75it [00:01, 38.24it/s]
80it [00:02, 38.01it/s]
85it [00:02, 38.47it/s]
90it [00:02, 38.66it/s]
95it [00:02, 38.42it/s]
100it [00:02, 37.54it/s]
104it [00:02, 37.60it/s]
109it [00:02, 37.95it/s]
114it [00:03, 37.11it/s]
118it [00:03, 36.94it/s]
123it [00:03, 37.15it/s]
128it [00:03, 36.85it/s]
132it [00:03, 36.58it/s]
137it [00:03, 36.92it/s]
141it [00:03, 36.18it/s]
145it [00:04, 35.99it/s]
149it [00:04, 35.59it/s]
153it [00:04, 34.56it/s]
157it [00:04, 34.41it/s]
160it [00:04, 33.99it/s]
164it [00:04, 34.09it/s]
171it [00:04, 34.78it/s]
176it [00:05, 34.53it/s]
180it [00:05, 34.23it/s]
184it [00:05, 34.23it/s]
188it [00:05, 33.72it/s]
191it [00:05, 33.42

...

4075it [01:55, 35.29it/s]
4081it [01:55, 35.29it/s]
4086it [01:55, 35.30it/s]
4091it [01:55, 35.31it/s]
4096it [01:56, 35.31it/s]
4101it [01:56, 35.29it/s]
4105it [01:56, 35.27it/s]
4109it [01:56, 35.27it/s]
4114it [01:56, 35.28it/s]
4118it [01:56, 35.28it/s]
4122it [01:56, 35.28it/s]
4126it [01:56, 35.27it/s]
4130it [01:57, 35.27it/s]
4134it [01:57, 35.26it/s]
4138it [01:57, 35.22it/s]
4142it [01:57, 35.22it/s]
4145it [01:57, 35.21it/s]
4152it [01:57, 35.24it/s]
4156it [01:57, 35.24it/s]
4160it [01:58, 35.23it/s]
4164it [01:58, 35.22it/s]
4168it [01:58, 35.22it/s]
4172it [01:58, 35.21it/s]
4176it [01:58, 35.20it/s]
4181it [01:58, 35.21it/s]
4186it [01:58, 35.21it/s]
4192it [01:58, 35.23it/s]
4197it [01:59, 35.23it/s]
4203it [01:59, 35.25it/s]
4208it [01:59, 35.26it/s]
4213it [01:59, 35.25it/s]
4218it [01:59, 35.26it/s]
4223it [01:59, 35.26it/s]
4228it [01:59, 35.25it/s]
4232it [02:00, 35.25it/s]
4236it [02:00, 35.24it/s]
4241it [02:00, 35.22it/s]
4247it [02:00, 35.24it/s]
4251it [02:0

5419it [02:32, 35.61it/s]
5425it [02:32, 35.62it/s]
5430it [02:32, 35.61it/s]
5434it [02:32, 35.61it/s]
5438it [02:32, 35.60it/s]
5442it [02:32, 35.60it/s]
5446it [02:33, 35.59it/s]
5450it [02:33, 35.59it/s]
5454it [02:33, 35.58it/s]
5458it [02:33, 35.58it/s]
5462it [02:33, 35.58it/s]
5466it [02:33, 35.58it/s]
5470it [02:33, 35.58it/s]
5475it [02:33, 35.58it/s]
5479it [02:34, 35.56it/s]
5483it [02:34, 35.56it/s]
5487it [02:34, 35.56it/s]
5492it [02:34, 35.56it/s]
5496it [02:34, 35.56it/s]
5500it [02:34, 35.55it/s]
5504it [02:34, 35.54it/s]
5508it [02:34, 35.54it/s]
5514it [02:35, 35.55it/s]
5518it [02:35, 35.55it/s]
5523it [02:35, 35.56it/s]
5527it [02:35, 35.55it/s]
5531it [02:35, 35.56it/s]
5537it [02:35, 35.57it/s]
5545it [02:35, 35.59it/s]
5551it [02:35, 35.60it/s]
5556it [02:36, 35.61it/s]
5561it [02:36, 35.61it/s]
5567it [02:36, 35.62it/s]
5574it [02:36, 35.64it/s]
5580it [02:36, 35.64it/s]
5585it [02:36, 35.64it/s]
5590it [02:36, 35.64it/s]
5595it [02:37, 35.63it/s]
5600it [02:3

6756it [03:10, 35.53it/s]
6761it [03:10, 35.54it/s]
6766it [03:10, 35.54it/s]
6771it [03:10, 35.55it/s]
6776it [03:10, 35.56it/s]
6781it [03:10, 35.55it/s]
6786it [03:10, 35.56it/s]
6791it [03:10, 35.56it/s]
6796it [03:11, 35.56it/s]
6802it [03:11, 35.58it/s]
6807it [03:11, 35.58it/s]
6812it [03:11, 35.58it/s]
6817it [03:11, 35.58it/s]
6822it [03:11, 35.59it/s]
6827it [03:11, 35.58it/s]
6832it [03:11, 35.59it/s]
6837it [03:12, 35.60it/s]
6842it [03:12, 35.58it/s]
6849it [03:12, 35.60it/s]
6854it [03:12, 35.60it/s]
6859it [03:12, 35.60it/s]
6864it [03:12, 35.59it/s]
6868it [03:12, 35.59it/s]
6872it [03:13, 35.59it/s]
6878it [03:13, 35.60it/s]
6885it [03:13, 35.62it/s]
6890it [03:13, 35.61it/s]
6895it [03:13, 35.61it/s]
6902it [03:13, 35.62it/s]
6907it [03:13, 35.63it/s]
6913it [03:13, 35.64it/s]
6919it [03:14, 35.65it/s]
6925it [03:14, 35.66it/s]
6931it [03:14, 35.65it/s]
6936it [03:14, 35.65it/s]
6941it [03:14, 35.65it/s]
6945it [03:14, 35.65it/s]
6950it [03:14, 35.66it/s]
6956it [03:1

8120it [03:47, 35.64it/s]
8124it [03:48, 35.63it/s]
8128it [03:48, 35.62it/s]
8133it [03:48, 35.63it/s]
8137it [03:48, 35.62it/s]
8141it [03:48, 35.62it/s]
8145it [03:48, 35.60it/s]
8148it [03:48, 35.59it/s]
8151it [03:49, 35.58it/s]
8155it [03:49, 35.58it/s]
8158it [03:49, 35.58it/s]
8164it [03:49, 35.58it/s]
8168it [03:49, 35.58it/s]
8172it [03:49, 35.58it/s]
8176it [03:49, 35.58it/s]
8180it [03:49, 35.58it/s]
8184it [03:50, 35.58it/s]
8188it [03:50, 35.58it/s]
8192it [03:50, 35.58it/s]
8197it [03:50, 35.58it/s]
8201it [03:50, 35.58it/s]
8205it [03:50, 35.57it/s]
8209it [03:50, 35.55it/s]
8214it [03:51, 35.55it/s]
8220it [03:51, 35.56it/s]
8225it [03:51, 35.56it/s]
8232it [03:51, 35.58it/s]
8238it [03:51, 35.59it/s]
8243it [03:51, 35.59it/s]
8249it [03:51, 35.60it/s]
8255it [03:51, 35.61it/s]
8261it [03:51, 35.62it/s]
8267it [03:52, 35.62it/s]
8273it [03:52, 35.63it/s]
8278it [03:52, 35.62it/s]
8283it [03:52, 35.62it/s]
8287it [03:52, 35.61it/s]
8292it [03:52, 35.62it/s]
8296it [03:5

scaling Data
total running time : 0:04:25.232707


In [22]:
classifierList = glob(classifierPath+'*'+modelName)

In [23]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

NeuralNetwork_2
XGBoost
RandomForestClassifier
LogisticRegression
SVC
NeuralNetwork_1


In [24]:
%%time
warnings.filterwarnings('ignore')
predictOutcome = dict(map(lambda x: bm.PredictNewsClassification(vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome = pd.DataFrame.from_dict(predictOutcome)
predictOutcome = predictOutcome.applymap(lambda x: le.inverse_transform(int(x)))
predictOutcome = extDaumData.merge(predictOutcome,
                                   left_index = True, right_index = True)
predictOutcome.to_csv('./outcome/outcome_news_classification_'+modelName,index=None, encoding='utf-8')


0it [00:00, ?it/s]
9372it [00:00, 551365.01it/s]
0it [00:00, ?it/s]
9372it [00:00, 667329.04it/s]
0it [00:00, ?it/s]
9372it [00:00, 672592.86it/s]

CPU times: user 33.7 s, sys: 542 ms, total: 34.3 s
Wall time: 34 s


#### Model3

In [25]:
wv1, vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 500, daumData2)
modelName = bm.Return_ModelName('fastText', model3,'ct')


 10%|█         | 8070/80610 [00:00<00:00, 79806.17it/s]

FastText(vocab=80610, size=500, alpha=0.025)



100%|██████████| 80610/80610 [00:01<00:00, 78701.49it/s]
0it [00:00, ?it/s]
2it [00:00, 17.49it/s]

running time : 0:00:01.031335
Vectorizing Data



5it [00:00, 20.49it/s]
8it [00:00, 22.18it/s]
11it [00:00, 23.82it/s]
14it [00:00, 24.57it/s]
17it [00:00, 25.09it/s]
20it [00:00, 25.43it/s]
24it [00:00, 26.87it/s]
28it [00:00, 28.14it/s]
32it [00:01, 28.93it/s]
36it [00:01, 26.98it/s]
39it [00:01, 26.21it/s]
43it [00:01, 27.00it/s]
47it [00:01, 27.72it/s]
51it [00:01, 27.88it/s]
57it [00:01, 29.46it/s]
61it [00:02, 29.44it/s]
68it [00:02, 31.13it/s]
73it [00:02, 31.48it/s]
78it [00:02, 31.87it/s]
83it [00:02, 32.44it/s]
88it [00:02, 33.03it/s]
93it [00:02, 32.54it/s]
97it [00:02, 32.69it/s]
101it [00:03, 31.96it/s]
107it [00:03, 32.65it/s]
111it [00:03, 32.67it/s]
115it [00:03, 32.15it/s]
119it [00:03, 32.27it/s]
123it [00:03, 32.43it/s]
128it [00:03, 32.60it/s]
132it [00:04, 32.58it/s]
137it [00:04, 32.98it/s]
141it [00:04, 32.66it/s]
145it [00:04, 32.66it/s]
149it [00:04, 32.50it/s]
153it [00:04, 31.86it/s]
156it [00:04, 31.77it/s]
159it [00:05, 31.13it/s]
163it [00:05, 31.28it/s]
169it [00:05, 31.79it/s]
173it [00:05, 31.62it/s]

...

4627it [02:42, 28.43it/s]
4632it [02:42, 28.44it/s]
4636it [02:43, 28.43it/s]
4640it [02:43, 28.43it/s]
4645it [02:43, 28.44it/s]
4650it [02:43, 28.45it/s]
4656it [02:43, 28.47it/s]
4660it [02:43, 28.47it/s]
4669it [02:43, 28.51it/s]
4675it [02:43, 28.52it/s]
4681it [02:44, 28.54it/s]
4687it [02:44, 28.55it/s]
4693it [02:44, 28.56it/s]
4699it [02:44, 28.58it/s]
4704it [02:44, 28.58it/s]
4709it [02:44, 28.58it/s]
4714it [02:44, 28.59it/s]
4718it [02:45, 28.59it/s]
4722it [02:45, 28.59it/s]
4726it [02:45, 28.59it/s]
4733it [02:45, 28.61it/s]
4738it [02:45, 28.62it/s]
4743it [02:45, 28.60it/s]
4747it [02:45, 28.60it/s]
4751it [02:46, 28.61it/s]
4755it [02:46, 28.61it/s]
4760it [02:46, 28.63it/s]
4764it [02:46, 28.63it/s]
4768it [02:46, 28.63it/s]
4772it [02:46, 28.63it/s]
4777it [02:46, 28.64it/s]
4783it [02:46, 28.66it/s]
4789it [02:47, 28.67it/s]
4795it [02:47, 28.69it/s]
4802it [02:47, 28.71it/s]
4808it [02:47, 28.73it/s]
4815it [02:47, 28.75it/s]
4821it [02:47, 28.76it/s]
4827it [02:4

6052it [03:20, 30.25it/s]
6059it [03:20, 30.27it/s]
6065it [03:20, 30.27it/s]
6071it [03:20, 30.28it/s]
6077it [03:20, 30.29it/s]
6082it [03:20, 30.30it/s]
6087it [03:21, 30.28it/s]
6092it [03:21, 30.29it/s]
6097it [03:21, 30.30it/s]
6102it [03:21, 30.29it/s]
6106it [03:21, 30.29it/s]
6110it [03:21, 30.29it/s]
6114it [03:21, 30.29it/s]
6119it [03:21, 30.30it/s]
6124it [03:22, 30.31it/s]
6129it [03:22, 30.32it/s]
6135it [03:22, 30.33it/s]
6140it [03:22, 30.34it/s]
6145it [03:22, 30.34it/s]
6150it [03:22, 30.34it/s]
6154it [03:22, 30.35it/s]
6160it [03:22, 30.36it/s]
6165it [03:23, 30.36it/s]
6170it [03:23, 30.37it/s]
6175it [03:23, 30.38it/s]
6181it [03:23, 30.39it/s]
6186it [03:23, 30.40it/s]
6192it [03:23, 30.41it/s]
6198it [03:23, 30.42it/s]
6204it [03:23, 30.43it/s]
6210it [03:24, 30.44it/s]
6215it [03:24, 30.44it/s]
6220it [03:24, 30.44it/s]
6224it [03:24, 30.44it/s]
6228it [03:24, 30.43it/s]
6232it [03:24, 30.43it/s]
6237it [03:24, 30.44it/s]
6241it [03:25, 30.44it/s]
6245it [03:2

7441it [03:58, 31.25it/s]
7445it [03:58, 31.25it/s]
7449it [03:58, 31.24it/s]
7455it [03:58, 31.25it/s]
7459it [03:58, 31.25it/s]
7463it [03:58, 31.25it/s]
7467it [03:58, 31.25it/s]
7471it [03:59, 31.25it/s]
7476it [03:59, 31.25it/s]
7481it [03:59, 31.26it/s]
7487it [03:59, 31.27it/s]
7493it [03:59, 31.28it/s]
7498it [03:59, 31.29it/s]
7503it [03:59, 31.29it/s]
7509it [03:59, 31.30it/s]
7518it [04:00, 31.32it/s]
7524it [04:00, 31.33it/s]
7530it [04:00, 31.32it/s]
7536it [04:00, 31.33it/s]
7541it [04:00, 31.34it/s]
7546it [04:00, 31.34it/s]
7551it [04:01, 31.33it/s]
7555it [04:01, 31.33it/s]
7559it [04:01, 31.33it/s]
7563it [04:01, 31.32it/s]
7567it [04:01, 31.32it/s]
7571it [04:01, 31.32it/s]
7577it [04:01, 31.34it/s]
7581it [04:01, 31.33it/s]
7585it [04:02, 31.34it/s]
7589it [04:02, 31.33it/s]
7593it [04:02, 31.33it/s]
7597it [04:02, 31.33it/s]
7601it [04:02, 31.33it/s]
7605it [04:02, 31.33it/s]
7609it [04:02, 31.33it/s]
7613it [04:02, 31.33it/s]
7618it [04:03, 31.34it/s]
7622it [04:0

8862it [04:36, 32.09it/s]
8867it [04:36, 32.09it/s]
8872it [04:36, 32.10it/s]
8877it [04:36, 32.09it/s]
8882it [04:36, 32.09it/s]
8886it [04:36, 32.09it/s]
8890it [04:37, 32.09it/s]
8894it [04:37, 32.09it/s]
8898it [04:37, 32.09it/s]
8903it [04:37, 32.09it/s]
8907it [04:37, 32.09it/s]
8911it [04:37, 32.09it/s]
8917it [04:37, 32.10it/s]
8921it [04:37, 32.10it/s]
8925it [04:38, 32.10it/s]
8929it [04:38, 32.08it/s]
8934it [04:38, 32.09it/s]
8938it [04:38, 32.09it/s]
8944it [04:38, 32.09it/s]
8951it [04:38, 32.10it/s]
8956it [04:39, 32.10it/s]
8961it [04:39, 32.10it/s]
8965it [04:39, 32.10it/s]
8969it [04:39, 32.10it/s]
8973it [04:39, 32.10it/s]
8978it [04:39, 32.11it/s]
8982it [04:39, 32.11it/s]
8989it [04:39, 32.12it/s]
8997it [04:39, 32.14it/s]
9003it [04:40, 32.15it/s]
9009it [04:40, 32.14it/s]
9014it [04:40, 32.14it/s]
9020it [04:40, 32.15it/s]
9025it [04:40, 32.16it/s]
9030it [04:40, 32.16it/s]
9035it [04:40, 32.16it/s]
9040it [04:41, 32.16it/s]
9044it [04:41, 32.15it/s]
9048it [04:4

scaling Data
total running time : 0:04:51.990420


In [26]:
classifierList = glob(classifierPath+'*'+modelName)

In [27]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

NeuralNetwork_2
XGBoost
LogisticRegression
RandomForestClassifier
SVC
NeuralNetwork_1


In [28]:
%%time
warnings.filterwarnings('ignore')
predictOutcome = dict(map(lambda x: bm.PredictNewsClassification(vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome = pd.DataFrame.from_dict(predictOutcome)
predictOutcome = predictOutcome.applymap(lambda x: le.inverse_transform(int(x)))
predictOutcome = extDaumData.merge(predictOutcome,
                                   left_index = True, right_index = True)
predictOutcome.to_csv('./outcome/outcome_news_classification_'+modelName,index=None, encoding='utf-8')


0it [00:00, ?it/s]
9372it [00:00, 604958.86it/s]
0it [00:00, ?it/s]
9372it [00:00, 536532.00it/s]
0it [00:00, ?it/s]
9372it [00:00, 727863.89it/s]

CPU times: user 34.8 s, sys: 476 ms, total: 35.3 s
Wall time: 34.6 s


### Mecab

#### News to Tagged Document

In [31]:
if os.path.isfile('./data/pre_data/news_tagged_data/pre_data_daum_news_by_mecab_for_fastText_news_classification.pickled'):
    daumData2 = pickle.load(open('./data/pre_data/news_tagged_data/pre_data_daum_news_by_mecab_for_fastText_news_classification.pickled', 'rb'))
else:
    daumData2 = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, mecab, stopwords, 'daum')
    pickle.dump(daumData2, open('./data/pre_data/news_tagged_data/pre_data_daum_news_by_mecab_for_fastText_news_classification.pickled', 'wb'))


  6%|▌         | 529/9372 [00:22<06:21, 23.18it/s]


 14%|█▎        | 1268/9372 [00:44<04:42, 28.70it/s]


 22%|██▏       | 2054/9372 [01:04<03:51, 31.62it/s]


 30%|███       | 2820/9372 [01:26<03:20, 32.62it/s]


 38%|███▊      | 3576/9372 [01:46<02:52, 33.67it/s]


...

 74%|███████▍  | 6916/9372 [03:04<01:05, 37.50it/s]


 82%|████████▏ | 7676/9372 [03:24<00:45, 37.62it/s]


 91%|█████████ | 8499/9372 [03:43<00:22, 37.96it/s]


 99%|█████████▉| 9273/9372 [04:03<00:02, 38.07it/s]


100%|██████████| 9372/9372 [04:05<00:00, 38.12it/s]

#### Load Model

In [ ]:
model1 = FastText.load(loadModelPath+'fastText_size-500_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-mecab.model')
model2 = FastText.load(loadModelPath+'fastText_size-500_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-mecab.model')
model3 = FastText.load(loadModelPath+'fastText_size-500_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-mecab.model')

#### Model1

In [ ]:
wv1, vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 500, daumData2)
modelName = bm.Return_ModelName('fastText', model1,'mecab')

  4%|▍         | 3072/80210 [00:00<00:02, 30711.68it/s]

FastText(vocab=80210, size=500, alpha=0.025)


100%|██████████| 80210/80210 [00:00<00:00, 95270.78it/s]
1it [00:00,  5.49it/s]

running time : 0:00:00.852679
Vectorizing Data


8395it [03:45, 37.17it/s]

In [ ]:
classifierList = glob(classifierPath+'*'+modelName)

In [ ]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

In [ ]:
%%time
warnings.filterwarnings('ignore')
predictOutcome = dict(map(lambda x: bm.PredictNewsClassification(vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome = pd.DataFrame.from_dict(predictOutcome)
predictOutcome = predictOutcome.applymap(lambda x: le.inverse_transform(int(x)))
predictOutcome = extDaumData.merge(predictOutcome,
                                   left_index = True, right_index = True)
predictOutcome.to_csv('./outcome/outcome_news_classification_'+modelName,index=None, encoding='utf-8')

#### Model2

In [ ]:
wv1, vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 500, daumData2)
modelName = bm.Return_ModelName('fastText', model2,'mecab')

In [ ]:
classifierList = glob(classifierPath+'*'+modelName)

In [ ]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

In [ ]:
%%time
warnings.filterwarnings('ignore')
predictOutcome = dict(map(lambda x: bm.PredictNewsClassification(vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome = pd.DataFrame.from_dict(predictOutcome)
predictOutcome = predictOutcome.applymap(lambda x: le.inverse_transform(int(x)))
predictOutcome = extDaumData.merge(predictOutcome,
                                   left_index = True, right_index = True)
predictOutcome.to_csv('./outcome/outcome_news_classification_'+modelName,index=None, encoding='utf-8')

#### Model3

In [ ]:
wv1, vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 500, daumData2)
modelName = bm.Return_ModelName('fastText', model3,'mecab')

In [ ]:
classifierList = glob(classifierPath+'*'+modelName)

In [ ]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

In [ ]:
%%time
warnings.filterwarnings('ignore')
predictOutcome = dict(map(lambda x: bm.PredictNewsClassification(vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome = pd.DataFrame.from_dict(predictOutcome)
predictOutcome = predictOutcome.applymap(lambda x: le.inverse_transform(int(x)))
predictOutcome = extDaumData.merge(predictOutcome,
                                   left_index = True, right_index = True)
predictOutcome.to_csv('./outcome/outcome_news_classification_'+modelName,index=None, encoding='utf-8')